In [1]:
%load_ext lab_black

Step: Load data

In [2]:
import csv
import pandas as pd
import numpy as np
import time

import os


from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.pipeline.core import Pipeline, ScheduleRecurrence, Schedule
from azureml.pipeline.steps import PythonScriptStep

from azureml.core import Workspace, Dataset

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline, PipelineData, PublishedPipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

from azureml.core import Workspace, Dataset, Datastore

workspace = Workspace.from_config()

datastore = workspace.get_default_datastore()

In [3]:
environment = Environment.from_conda_specification("nfl_env", "nfl_env.yaml")

In [4]:
# Create a compute target
compute_name = "matthewhanauer991"
# Define the environment and dependencies for your script


try:
    compute_target = ComputeTarget(workspace=workspace, name=compute_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

In [5]:
import os

script_path = os.path.abspath("production_injury_data.py")

src = ScriptRunConfig(source_directory=".", script=script_path, environment=environment)

In [6]:
# Create a ScriptRunConfig for your script
src = ScriptRunConfig(
    source_directory=".", script="production_injury_data.py", environment=environment
)

In [7]:
# Create a PythonScriptStep
step = PythonScriptStep(
    name="scrapper_nfl_injury",
    script_name="production_injury_data.py",
    compute_target=compute_target,
    runconfig=src.run_config,
)

In [8]:
# Create a Pipeline using the PythonScriptStep
pipeline = Pipeline(workspace=workspace, steps=[step])

In [9]:
# Publish the pipeline
published_pipeline = pipeline.publish(name="scrapper_nfl_injury_pipeline")

Created step scrapper_nfl_injury [4b55e79c][631dbd59-3801-4862-8a2f-98ef3dd3e0d3], (This step will run and generate new outputs)


In [10]:
from datetime import time


recurrence = ScheduleRecurrence(frequency="Day", interval=1, time_of_day="15:45")
schedule = Schedule.create(
    workspace,
    name="scrapper_nfl_injury_schedule",
    pipeline_id=published_pipeline.id,
    experiment_name="scrapper_nfl_injury_experiment",
    recurrence=recurrence,
)